# Travail autour de la pauvreté

## Librairies et initialisations nécessaires

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

!pip install dbfread folium geopandas geojson

import geopandas as gpd
import pandas as pd
import folium
import json
import requests
from dbfread import DBF
def conv_df(chemin):
  gdf = gpd.GeoDataFrame(gpd.read_file(chemin, geometry='geometry'))
  return gdf

## Analyse descriptive des données sur les QPV de l'année 2017

### Sur les revenus déclarés

#### Récupération des données

In [ ]:
# Les données de notre DataFrame
df_pauvrete_2017 = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2017_csv/data_filo17_qp_revdec.csv')

# Explication des variables du DatFrame
meta_pauvrete_2017 = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2017_csv/meta_filo17_qp_revdec.csv')

#### Observation des données récupérées

In [ ]:
# Observation des données récupérées du dossier csv 'data_filo17_qp_revdec.csv'
df_pauvrete_2017.head()

In [ ]:
# Description du jeu de données
df_pauvrete_2017.describe()

In [ ]:
# Affichage du nombre de QPV
n = len(df_pauvrete_2017['decuc_q1_a17'])
print("Le nombre de QPV étudiés : ", n)

In [ ]:
# Calcul du nombre de données qui peuvent être exploitées en se basant sur la note de précaution d'utilisation des données dans le DataFrame

# initialisation des compteurs
count_0, count_1, count_5 = 0, 0, 0

# remplissage des conteurs
for val in df_pauvrete_2017['note']:
  # Les données sont exploitables
  if val == '0':
    count_0 += 1
  # Les données sont à exploiter avec précaution
  elif val == '1':
    count_1 += 1
  # Les données sont non diffusées
  else :
    count_5 += 1

# Affichage des résultats obtenus
print("Le nombre de données n'ayant aucun soucis : {} ({:.2f}%)".format(count_0, count_0/n*100))

print("Le nombre de données à utiliser avec précaution : {} ({:.2f}%)".format(count_1, count_1/n*100))
print("Le nombre de données non diffusées : {} ({:.2f}%)".format(count_5, count_5/n*100))

In [ ]:
# Affichage des QPV à utiliser avec précaution

# Récupération des données à exploiter avec précaution (la note vaut 1)
df_precaution = df_pauvrete_2017[df_pauvrete_2017['note']=='1']

# Récupération du code du QPV
df_precaution = df_precaution['codgeo']

# Affichage des QPV
df_precaution

#### Traitement des données pour récupérer seulement celles nécessaires

In [ ]:
# Retirer les éléments qui sont à traiter avec précaution pour avoir des données exploitables

# Initialisation des variables nous permettant de traverser les quartiers
ind_codgeo = 0
ind_codgeo_precaution = 0
codgeo_precaution = df_precaution.iloc[0]

# Boucle pour éliminer les QPV à étudier avec précaution
for codgeo in df_pauvrete_2017['codgeo']:
  if (codgeo_precaution == codgeo):
    df_pauvrete_2017 = df_pauvrete_2017.drop(index = ind_codgeo)
    ind_codgeo_precaution += 1

    # Dans le cas où nous avons éliminé tout les QPV que nous ne voulons plus
    if ind_codgeo_precaution >= len(df_precaution):
      break
    codgeo_precaution = df_precaution.iloc[ind_codgeo_precaution]

  # Incrémentation pour passer au quartier suivant
  ind_codgeo += 1

In [ ]:
# Afficher les données manquantes

#Calcul du nombre de nan
for colonne in df_pauvrete_2017.columns:
  nom_colonne=colonne
  nombre_nan = df_pauvrete_2017[nom_colonne].isna().sum()
  dif=len(df_pauvrete_2017)-nombre_nan
  print(f"Le nombre de NaN dans la colonne '{nom_colonne}' est : {nombre_nan} '. Nombre de valeurs:'{dif}")

#### Graphique

In [ ]:
# Calcul du nombre de QPV par département en récupérant le code pour le prochain graphique

# Le calcul du nombre de QPV par département
QPV = []
for codgeo in df_pauvrete_2017['codgeo']:
  # Dans le cas de la première ligne
  if codgeo[2] == 's':
    continue
  # Pour garder seulement les départements métropolitains
  elif codgeo[2] != '0' :
    break
  # Ajout du département où appartient le quartier
  QPV.append(codgeo[3:5])


# Initialisation de la variable qui va récupérer tout les départements
dep = []

# Récupération du nombre de QPV par département
nb_QPV_par_dep = []
for i in range(95):
  if i+1 < 10 :
    dep.append('0' + str(i+1))
  else :
    # Le cas des deux départements Corse
    if i == 30 :
      dep.append('2A')
      nb_QPV_par_dep.append(QPV.count(dep[i]))
      dep.append('2B')
      nb_QPV_par_dep.append(QPV.count(dep[i]))
    dep.append(str(i+1))
  # Ajout du nombre de QPV dans l'ordre des départements
  nb_QPV_par_dep.append(QPV.count(dep[i]))


In [ ]:
# Affichage du diagramme
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 6))

plt.bar(dep, nb_QPV_par_dep)

plt.title("Répartition des QPV analysés par département", fontsize=14)
plt.xlabel("Département", fontsize=12)
plt.ylabel("Nombre de QPV", fontsize=12)
plt.xticks(rotation=45, fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Inégalité des ménages avec l'indice de Gini

In [ ]:
# Le département ayant les plus fortes inégalités dans les QPV
max_inegalite = max(df_pauvrete_2017['decuc_gini_a17'])
print('QPV ayant le plus d inégalités')
df_pauvrete_2017[df_pauvrete_2017['decuc_gini_a17']==max_inegalite]

In [ ]:
# Le département ayant les plus faibles inégalités dans les QPV
min_inegalite = min(df_pauvrete_2017['decuc_gini_a17'])
print('QPV ayant le moins d inégalités')
df_pauvrete_2017[df_pauvrete_2017['decuc_gini_a17']==min_inegalite]

### Les revenus disponibles

#### Récupération des donneés

In [ ]:
# Les données de notre DataFrame
df_pauvrete_2017_dis = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2017_csv/data_filo17_qp_revdis.csv')

# Explication des variables
meta_pauvrete_2017_dis = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2017_csv/meta_filo17_qp_revdis.csv')

#### Observation des données

In [ ]:
# Affichage des données présentes dans notre DataFrame
df_pauvrete_2017_dis.head()

In [ ]:
# Rapide analyse descriptive des données présentes dans notre DataFrame
df_pauvrete_2017_dis.describe()

#### Les inégalités des ménages dans les QPV avec l'indice de Gini

In [ ]:
# Le département ayant les plus fortes inégalités dans les QPV
max_inegalite = max(df_pauvrete_2017_dis['disp_gini_a17'])
print('QPV ayant le plus d inégalités')
df_pauvrete_2017_dis[df_pauvrete_2017_dis['disp_gini_a17']==max_inegalite]

In [ ]:
# Le département ayant les plus faibles inégalités dans les QPV
min_inegalite = min(df_pauvrete_2017_dis['disp_gini_a17'])
print('QPV ayant le moins d inégalités')
df_pauvrete_2017_dis[df_pauvrete_2017_dis['disp_gini_a17']==min_inegalite]

## Analyse descriptive de la pauvreté par département dans les QPV de 2017 à 2021

In [ ]:
# Choix du département
dep = '30'

### Les revenus déclarés

#### Récupération des données

In [ ]:
# 2017
df_pauvrete_2017_dec = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2017_csv/data_filo17_qp_revdec.csv')

# 2018
df_pauvrete_2018_dec = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2018_csv/data_filo18_qp_revdec.csv')

# 2019
df_pauvrete_2019_dec = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2019_csv/data_filo19_qp_revdec.csv')

#2020
df_pauvrete_2020_dec = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_qpv_2020_csv/data_filo20_qp_revdec.csv')

#2021
df_pauvrete_2021_dec = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2021_qp15_csv/data_indic-revenu-declare-qp15-2021_QP.csv')

# Suppression des données ayant une note différente de 0 ce qui signifie que les données sont à exploiter avec précaution

#######################
# Les données de 2017 #
#######################

# Remplace les valeurs vides par des valeurs nulles
df_pauvrete_2017_dec.replace(["", " "], pd.NA, inplace=True)

# Boucle permettant de récupérer le nombre de données manquantes et de récupérer seulement les données exploitables du département choisi préalablement
ind = 0
missing_data_2017 = 0
df_dep_2017_dec = []
for i in range(len(df_pauvrete_2017_dec)):
  QPV = df_pauvrete_2017_dec.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[3:5] == dep and QPV['note']=='0':
    df_dep_2017_dec.append(QPV)
    missing_data_2017 += QPV.isna().sum()
    ind = -1

  # Nous avons récupére les données nécessaires du département donc nous pouvons arrêter la boucle
  elif ind == -1 and codgeo[3:5] != dep:
    break


#######################
# Les données de 2018 #
#######################

# Remplace les valeurs vides par des valeurs nulles
df_pauvrete_2018_dec.replace(["", " "], pd.NA, inplace=True)

# Boucle permettant de récupérer le nombre de données manquantes et de récupérer seulement les données exploitables du département choisi préalablement
ind = 0
missing_data_2018 = 0
df_dep_2018_dec = []
for i in range(len(df_pauvrete_2018_dec)):
  QPV = df_pauvrete_2018_dec.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[3:5] == dep and QPV['DEC_NOTE_A18']=='0':
    df_dep_2018_dec.append(QPV)
    missing_data_2018 += QPV.isna().sum()
    ind = -1

  # Nous avons récupére les données nécessaires du département donc nous pouvons arrêter la boucle
  elif ind == -1 and codgeo[3:5] != dep:
    break


#######################
# Les données de 2019 #
#######################

# Remplace les valeurs vides par des valeurs nulles
df_pauvrete_2019_dec.replace(["", " "], pd.NA, inplace=True)

# Boucle permettant de récupérer le nombre de données manquantes et de récupérer seulement les données exploitables du département choisi préalablement
ind = 0
missing_data_2019 = 0
df_dep_2019_dec = []
for i in range(len(df_pauvrete_2019_dec)):
  QPV = df_pauvrete_2019_dec.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[3:5] == dep and QPV['DEC_NOTE_A19']=='0':
    df_dep_2019_dec.append(QPV)
    missing_data_2019 += QPV.isna().sum()
    ind = -1

  # Nous avons récupére les données nécessaires du département donc nous pouvons arrêter la boucle
  elif ind == -1 and codgeo[3:5] != dep:
    break


#######################
# Les données de 2020 #
#######################

# Remplace les valeurs vides par des valeurs nulles
df_pauvrete_2020_dec.replace(["", " "], pd.NA, inplace=True)

# Boucle permettant de récupérer le nombre de données manquantes et de récupérer seulement les données exploitables du département choisi préalablement
ind = 0
missing_data_2020 = 0
df_dep_2020_dec = []
for i in range(len(df_pauvrete_2020_dec)):
  QPV = df_pauvrete_2020_dec.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[3:5] == dep and QPV['DEC_NOTE_A20']=='0':
    df_dep_2020_dec.append(QPV)
    missing_data_2020 += QPV.isna().sum()
    ind = -1

  # Nous avons récupére les données nécessaires du département donc nous pouvons arrêter la boucle
  elif ind == -1 and codgeo[3:5] != dep:
    break


#######################
# Les données de 2021 #
#######################

# Remplace les valeurs vides par des valeurs nulles
df_pauvrete_2021_dec.replace(["", " "], pd.NA, inplace=True)

# Boucle permettant de récupérer le nombre de données manquantes et de récupérer seulement les données exploitables du département choisi préalablement
ind = 0
missing_data_2021 = 0
df_dep_2021_dec = []
for i in range(len(df_pauvrete_2021_dec)):
  QPV = df_pauvrete_2021_dec.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[3:5] == dep and QPV['DEC_NOTE_A21']=='0':
    df_dep_2021_dec.append(QPV)
    missing_data_2021 += QPV.isna().sum()
    ind = -1

  # Nous avons récupére les données nécessaires du département donc nous pouvons arrêter la boucle
  elif ind == -1 and codgeo[3:5] != dep :
    break

#### Le nombre de QPV

In [ ]:
# Affichage du nombre de QPV étudié au cours des années dans le département selectionné

import matplotlib.pyplot as plt

# L'axe des abscisses
annee = ['2017', '2018', '2019', '2020', '2021']

# L'axe des ordonnées
QPV_par_annee = [len(df_dep_2017_dec), len(df_dep_2018_dec), len(df_dep_2019_dec), len(df_dep_2020_dec), len(df_dep_2021_dec)]

plt.figure(figsize=(16, 6))  # Augmenter la largeur de la figure

plt.bar(annee, QPV_par_annee)

plt.title("QPV analysés dans le département " + dep, fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("Nombre de QPV", fontsize=12)
plt.xticks(rotation=45, fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Les données manquantes

In [ ]:
# Affichage de l'évolution des données manquantes au fil des années dans le département selectionné

# Récupération des données calculées lors du nettoyage des données pour l'axe des ordonnées
missing_data = [missing_data_2017, missing_data_2018, missing_data_2019, missing_data_2020, missing_data_2021]

plt.figure(figsize=(16, 6))  # Augmenter la largeur de la figure

plt.plot(annee, missing_data)

plt.title("Données manquantes du département " + dep + " par année", fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("Nombre de données manquantes", fontsize=12)
plt.xticks(rotation=45, fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Le revenu déclaré moyen par unité de consommation (quartile)

In [ ]:
# Librairie nécessaire pour utiliser la fonction mean (pour calculer la moyenne)
import statistics

# Récupération pour chaque année du revenu au niveau des quartiles des ménages des QPV du département sélectionné

# 2017
premier_quartile_2017 = []
mediane_2017 = []
troisieme_quartile_2017 = []

for QPV in df_dep_2017_dec:
  if pd.notna(QPV.iloc[1]):
    valeur = QPV.iloc[1].replace('\xa0', '').replace(' ', '')
    premier_quartile_2017.append(int(valeur))

  if pd.notna(QPV.iloc[2]):
    valeur = QPV.iloc[2].replace('\xa0', '').replace(' ', '')
    mediane_2017.append(int(valeur))

  if pd.notna(QPV.iloc[3]):
    valeur = QPV.iloc[3].replace('\xa0', '').replace(' ', '')
    troisieme_quartile_2017.append(int(valeur))


# 2018
premier_quartile_2018 = []
mediane_2018 = []
troisieme_quartile_2018 = []
for QPV in df_dep_2018_dec:
  if pd.notna(QPV.iloc[1]):
    valeur = QPV.iloc[1].replace('\xa0', '').replace(' ', '')
    premier_quartile_2018.append(int(valeur))

  if pd.notna(QPV.iloc[2]):
    valeur = QPV.iloc[2].replace('\xa0', '').replace(' ', '')
    mediane_2018.append(int(valeur))

  if pd.notna(QPV.iloc[3]):
    valeur = QPV.iloc[3].replace('\xa0', '').replace(' ', '')
    troisieme_quartile_2018.append(int(valeur))


# 2019
premier_quartile_2019 = []
mediane_2019 = []
troisieme_quartile_2019 = []
for QPV in df_dep_2019_dec:
  if pd.notna(QPV.iloc[1]):
    valeur = QPV.iloc[1].replace('\xa0', '').replace(' ', '')
    premier_quartile_2019.append(int(valeur))

  if pd.notna(QPV.iloc[2]):
    valeur = QPV.iloc[2].replace('\xa0', '').replace(' ', '')
    mediane_2019.append(int(valeur))

  if pd.notna(QPV.iloc[3]):
    valeur = QPV.iloc[3].replace('\xa0', '').replace(' ', '')
    troisieme_quartile_2019.append(int(valeur))

# 2020
premier_quartile_2020 = []
mediane_2020 = []
troisieme_quartile_2020 = []
for QPV in df_dep_2020_dec:
  if pd.notna(QPV.iloc[1]):
    valeur = QPV.iloc[1].replace('\xa0', '').replace(' ', '')
    premier_quartile_2020.append(int(valeur))

  if pd.notna(QPV.iloc[2]):
    valeur = QPV.iloc[2].replace('\xa0', '').replace(' ', '')
    mediane_2020.append(int(valeur))

  if pd.notna(QPV.iloc[3]):
    valeur = QPV.iloc[3].replace('\xa0', '').replace(' ', '')
    troisieme_quartile_2020.append(int(valeur))


# 2021
premier_quartile_2021 = []
mediane_2021 = []
troisieme_quartile_2021 = []
for QPV in df_dep_2021_dec:
  if pd.notna(QPV.iloc[1]):
    valeur = QPV.iloc[1].replace('\xa0', '').replace(' ', '')
    premier_quartile_2021.append(int(valeur))

  if pd.notna(QPV.iloc[2]):
    valeur = QPV.iloc[2].replace('\xa0', '').replace(' ', '')
    mediane_2021.append(int(valeur))

  if pd.notna(QPV.iloc[3]):
    valeur = QPV.iloc[3].replace('\xa0', '').replace(' ', '')
    troisieme_quartile_2021.append(int(valeur))


# Récupération des quartiles qui permettront de réaliser le graphique pour visualiser l'évolution
quartile_1_dep = [statistics.mean(premier_quartile_2017), statistics.mean(premier_quartile_2018), statistics.mean(premier_quartile_2019), statistics.mean(premier_quartile_2020), statistics.mean(premier_quartile_2021)]
mediane_dep = [statistics.mean(mediane_2017),statistics.mean(mediane_2018), statistics.mean(mediane_2019), statistics.mean(mediane_2020), statistics.mean(mediane_2021)]
quartile_3_dep = [statistics.mean(troisieme_quartile_2017), statistics.mean(troisieme_quartile_2018), statistics.mean(troisieme_quartile_2019), statistics.mean(troisieme_quartile_2020), statistics.mean(troisieme_quartile_2021)]


In [ ]:
# Affichage du graphique permettant de visualiser l'évolution des revenus des QPV au cours des années
plt.figure(figsize=(16, 6))  # Augmenter la largeur de la figure

plt.plot(annee, quartile_1_dep, label ='1er quartile')
plt.plot(annee, mediane_dep, label = 'médiane')
plt.plot(annee, quartile_3_dep, label = '3ème quartile')

plt.title("Les revenus déclarés moyen du département " + dep, fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("Revenu (en euros)", fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.legend()

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Le revenu déclaré moyen par unité de consommation (décile)

In [ ]:
# Récupération pour chaque année du revenu au niveau des déciles des ménages des QPV du département sélectionné

# résultats
D1 = []
D2 = []
D3 = []
D4 = []
D6 = []
D7 = []
D8 = []
D9 = []


# temporaire
decile_1 = []
decile_2 = []
decile_3 = []
decile_4 = []
decile_6 = []
decile_7 = []
decile_8 = []
decile_9 = []

# 2017

for QPV in df_dep_2017_dec:
  if pd.notna(QPV.iloc[5]):
    valeur = QPV.iloc[5].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(QPV.iloc[6]):
    valeur = QPV.iloc[6].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(QPV.iloc[7]):
    valeur = QPV.iloc[7].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(QPV.iloc[8]):
    valeur = QPV.iloc[8].replace('\xa0', '').replace(' ', '')
    decile_4.append(int(valeur))

  if pd.notna(QPV.iloc[9]):
    valeur = QPV.iloc[9].replace('\xa0', '').replace(' ', '')
    decile_6.append(int(valeur))

  if pd.notna(QPV.iloc[10]):
    valeur = QPV.iloc[10].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(QPV.iloc[11]):
    valeur = QPV.iloc[11].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(QPV.iloc[12]):
    valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

D1.append(statistics.mean(decile_1))
D2.append(statistics.mean(decile_2))
D3.append(statistics.mean(decile_3))
D4.append(statistics.mean(decile_4))
D6.append(statistics.mean(decile_6))
D7.append(statistics.mean(decile_7))
D8.append(statistics.mean(decile_8))
D9.append(statistics.mean(decile_9))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_4.clear()
decile_6.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()



# 2018
premier_quartile_2018 = []
mediane_2018 = []
troisieme_quartile_2018 = []
for QPV in df_dep_2018_dec:
  if pd.notna(QPV.iloc[5]):
    valeur = QPV.iloc[5].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(QPV.iloc[6]):
    valeur = QPV.iloc[6].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(QPV.iloc[7]):
    valeur = QPV.iloc[7].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(QPV.iloc[8]):
    valeur = QPV.iloc[8].replace('\xa0', '').replace(' ', '')
    decile_4.append(int(valeur))

  if pd.notna(QPV.iloc[9]):
    valeur = QPV.iloc[9].replace('\xa0', '').replace(' ', '')
    decile_6.append(int(valeur))

  if pd.notna(QPV.iloc[10]):
    valeur = QPV.iloc[10].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(QPV.iloc[11]):
    valeur = QPV.iloc[11].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(QPV.iloc[12]):
    valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

D1.append(statistics.mean(decile_1))
D2.append(statistics.mean(decile_2))
D3.append(statistics.mean(decile_3))
D4.append(statistics.mean(decile_4))
D6.append(statistics.mean(decile_6))
D7.append(statistics.mean(decile_7))
D8.append(statistics.mean(decile_8))
D9.append(statistics.mean(decile_9))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_4.clear()
decile_6.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()


# 2019
premier_quartile_2019 = []
mediane_2019 = []
troisieme_quartile_2019 = []
for QPV in df_dep_2019_dec:
  if pd.notna(QPV.iloc[5]):
    valeur = QPV.iloc[5].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(QPV.iloc[6]):
    valeur = QPV.iloc[6].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(QPV.iloc[7]):
    valeur = QPV.iloc[7].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(QPV.iloc[8]):
    valeur = QPV.iloc[8].replace('\xa0', '').replace(' ', '')
    decile_4.append(int(valeur))

  if pd.notna(QPV.iloc[9]):
    valeur = QPV.iloc[9].replace('\xa0', '').replace(' ', '')
    decile_6.append(int(valeur))

  if pd.notna(QPV.iloc[10]):
    valeur = QPV.iloc[10].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(QPV.iloc[11]):
    valeur = QPV.iloc[11].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(QPV.iloc[12]):
    valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

D1.append(statistics.mean(decile_1))
D2.append(statistics.mean(decile_2))
D3.append(statistics.mean(decile_3))
D4.append(statistics.mean(decile_4))
D6.append(statistics.mean(decile_6))
D7.append(statistics.mean(decile_7))
D8.append(statistics.mean(decile_8))
D9.append(statistics.mean(decile_9))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_4.clear()
decile_6.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()

# 2020
premier_quartile_2020 = []
mediane_2020 = []
troisieme_quartile_2020 = []
for QPV in df_dep_2020_dec:
  if pd.notna(QPV.iloc[5]):
    valeur = QPV.iloc[5].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(QPV.iloc[6]):
    valeur = QPV.iloc[6].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(QPV.iloc[7]):
    valeur = QPV.iloc[7].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(QPV.iloc[8]):
    valeur = QPV.iloc[8].replace('\xa0', '').replace(' ', '')
    decile_4.append(int(valeur))

  if pd.notna(QPV.iloc[9]):
    valeur = QPV.iloc[9].replace('\xa0', '').replace(' ', '')
    decile_6.append(int(valeur))

  if pd.notna(QPV.iloc[10]):
    valeur = QPV.iloc[10].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(QPV.iloc[11]):
    valeur = QPV.iloc[11].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(QPV.iloc[12]):
    valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

D1.append(statistics.mean(decile_1))
D2.append(statistics.mean(decile_2))
D3.append(statistics.mean(decile_3))
D4.append(statistics.mean(decile_4))
D6.append(statistics.mean(decile_6))
D7.append(statistics.mean(decile_7))
D8.append(statistics.mean(decile_8))
D9.append(statistics.mean(decile_9))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_4.clear()
decile_6.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()


# 2021
premier_quartile_2021 = []
mediane_2021 = []
troisieme_quartile_2021 = []
for QPV in df_dep_2021_dec:
  if pd.notna(QPV.iloc[5]):
    valeur = QPV.iloc[5].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(QPV.iloc[6]):
    valeur = QPV.iloc[6].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(QPV.iloc[7]):
    valeur = QPV.iloc[7].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(QPV.iloc[8]):
    valeur = QPV.iloc[8].replace('\xa0', '').replace(' ', '')
    decile_4.append(int(valeur))

  if pd.notna(QPV.iloc[9]):
    valeur = QPV.iloc[9].replace('\xa0', '').replace(' ', '')
    decile_6.append(int(valeur))

  if pd.notna(QPV.iloc[10]):
    valeur = QPV.iloc[10].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(QPV.iloc[11]):
    valeur = QPV.iloc[11].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(QPV.iloc[12]):
    valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

D1.append(statistics.mean(decile_1))
D2.append(statistics.mean(decile_2))
D3.append(statistics.mean(decile_3))
D4.append(statistics.mean(decile_4))
D6.append(statistics.mean(decile_6))
D7.append(statistics.mean(decile_7))
D8.append(statistics.mean(decile_8))
D9.append(statistics.mean(decile_9))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_4.clear()
decile_6.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()



In [ ]:
# Affichage du graphique montrant l'évolution des revenus par décile

plt.figure(figsize=(16, 6))  # Augmenter la largeur de la figure

# traçage de chaque décile
plt.plot(annee, D1, label='1er décile')
plt.plot(annee, D2, label='2ème décile')
plt.plot(annee, D3, label='3ème décile')
plt.plot(annee, D4, label='4ème décile')
plt.plot(annee, D6, label='6ème décile')
plt.plot(annee, D7, label='7ème décile')
plt.plot(annee, D8, label='8ème décile')
plt.plot(annee, D9, label='9ème décile')

plt.title("Les revenus déclarés moyen du département " + dep, fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("Revenu (en euros)", fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.legend()

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Indice de gini (les inégalités)

In [ ]:
# Récupération des données permettant d'observer l'évolution des inégalités dans le département selectionné en se servant de l'indice de Gini

import statistics

G = []
gini = []

# 2017
for QPV in df_dep_2017_dec:
  if pd.notna(QPV.iloc[14]):
    valeur = QPV.iloc[14].replace(',', '.')
    gini.append(float(valeur))

G.append(statistics.mean(gini))
gini.clear()


# 2018
for QPV in df_dep_2018_dec:
  if pd.notna(QPV.iloc[14]):
    valeur = QPV.iloc[14].replace(',', '.')
    gini.append(float(valeur))

G.append(statistics.mean(gini))
gini.clear()


# 2019
for QPV in df_dep_2019_dec:
  if pd.notna(QPV.iloc[14]):
    valeur = QPV.iloc[14].replace(',', '.')
    gini.append(float(valeur))

G.append(statistics.mean(gini))
gini.clear()


# 2020
for QPV in df_dep_2020_dec:
  if pd.notna(QPV.iloc[14]):
    valeur = QPV.iloc[14].replace(',', '.')
    gini.append(float(valeur))

G.append(statistics.mean(gini))
gini.clear()


# 2021
for QPV in df_dep_2021_dec:
  if pd.notna(QPV.iloc[14]):
    valeur = QPV.iloc[14].replace(',', '.')
    gini.append(float(valeur))

G.append(statistics.mean(gini))
gini.clear()


In [ ]:
# Affichage du graphique mettant en avant l'évolution des inégalités au sein des QPV dans le département selctionné de 2017 à 2021

plt.figure(figsize=(16, 6))

plt.plot(annee, G)

plt.title("Le niveau d'inégalité du département " + dep, fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("indice de Gini", fontsize=12)
plt.xticks(rotation=45, fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Rapport S80/S20

In [ ]:
# Récupération des données permettant d'observer l'évolution des inégalités au sein des QPV dans le département selectionné en se servant du Ratio s80/s20

import statistics

R_80_20 = []
ratio = []

# 2017
for QPV in df_dep_2017_dec:
  if pd.notna(QPV.iloc[15]):
    valeur = QPV.iloc[15].replace(',', '.')
    ratio.append(float(valeur))

R_80_20.append(statistics.mean(ratio))
ratio.clear()


# 2018
for QPV in df_dep_2018_dec:
  if pd.notna(QPV.iloc[15]):
    valeur = QPV.iloc[15].replace(',', '.')
    ratio.append(float(valeur))

R_80_20.append(statistics.mean(ratio))
ratio.clear()


# 2019
for QPV in df_dep_2019_dec:
  if pd.notna(QPV.iloc[15]):
    valeur = QPV.iloc[15].replace(',', '.')
    ratio.append(float(valeur))

R_80_20.append(statistics.mean(ratio))
ratio.clear()


# 2020
for QPV in df_dep_2020_dec:
  if pd.notna(QPV.iloc[15]):
    valeur = QPV.iloc[15].replace(',', '.')
    ratio.append(float(valeur))

R_80_20.append(statistics.mean(ratio))
ratio.clear()


# 2021
for QPV in df_dep_2021_dec:
  if pd.notna(QPV.iloc[15]):
    valeur = QPV.iloc[15].replace(',', '.')
    ratio.append(float(valeur))

R_80_20.append(statistics.mean(ratio))
ratio.clear()

In [ ]:
# Affichage du graphique permettant d'observer l'augmentation des inégalités qui est bien similaire au graphique précédent avec l'indice de Gini

plt.figure(figsize=(16, 6))

plt.plot(annee, R_80_20)

plt.title("Le niveau d'inégalité du département " + dep, fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("Ratio S80/S20", fontsize=12)
plt.xticks(rotation=45, fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### Les revenus disponibles

#### Récupération des données

In [ ]:
# 2017
df_pauvrete_2017_dis = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2017_csv/data_filo17_qp_revdis.csv')

# 2018
df_pauvrete_2018_dis = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2018_csv/data_filo18_qp_revdis.csv')

# 2019
df_pauvrete_2019_dis = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2019_csv/data_filo19_qp_revdis.csv')

#2020
df_pauvrete_2020_dis = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_qpv_2020_csv/data_filo20_qp_revdis.csv')

#2021
df_pauvrete_2021_dis = conv_df('/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/revenus_pauvrete_2021_qp15_csv/data_indic-revenu-disponible-qp15-2021_QP.csv')

In [ ]:
# Suppression des données ayant précautions d'exploitation en observant la variable note

# 2017
df_pauvrete_2017_dis.replace(["", " "], pd.NA, inplace=True)

ind = 0
missing_data_2017_dis = 0
df_dep_2017_dis = []
for i in range(len(df_pauvrete_2017_dis)):
  QPV = df_pauvrete_2017_dis.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[3:5] == dep and QPV['note']=='0':
    df_dep_2017_dis.append(QPV)
    missing_data_2017_dis += QPV.isna().sum()
    ind = -1

  elif ind == -1 and codgeo[3:5] != dep:
    break

# 2018
df_pauvrete_2018_dis.replace(["", " "], pd.NA, inplace=True)

ind = 0
missing_data_2018_dis = 0
df_dep_2018_dis = []
for i in range(len(df_pauvrete_2018_dis)):
  QPV = df_pauvrete_2018_dis.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[3:5] == dep and QPV['DISP_NOTE_A18']=='0':
    df_dep_2018_dis.append(QPV)
    missing_data_2018_dis += QPV.isna().sum()
    ind = -1

  elif ind == -1 and codgeo[3:5] != dep:
    break


# 2019
df_pauvrete_2019_dis.replace(["", " "], pd.NA, inplace=True)

ind = 0
missing_data_2019_dis = 0
df_dep_2019_dis = []
for i in range(len(df_pauvrete_2019_dis)):
  QPV = df_pauvrete_2019_dis.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[3:5] == dep and QPV['DISP_NOTE_A19']=='0':
    df_dep_2019_dis.append(QPV)
    missing_data_2019_dis += QPV.isna().sum()
    ind = -1

  elif ind == -1 and codgeo[3:5] != dep:
    break


# 2020
df_pauvrete_2020_dis.replace(["", " "], pd.NA, inplace=True)

ind = 0
missing_data_2020_dis = 0
df_dep_2020_dis = []
for i in range(len(df_pauvrete_2020_dis)):
  QPV = df_pauvrete_2020_dis.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[3:5] == dep and QPV['DISP_NOTE_A20']=='0':
    df_dep_2020_dis.append(QPV)
    missing_data_2020_dis += QPV.isna().sum()
    ind = -1

  elif ind == -1 and codgeo[3:5] != dep:
    break


# 2021
df_pauvrete_2021_dis.replace(["", " "], pd.NA, inplace=True)

ind = 0
missing_data_2021_dis = 0
df_dep_2021_dis = []
for i in range(len(df_pauvrete_2021_dis)):
  QPV = df_pauvrete_2021_dis.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[3:5] == dep and QPV['DISP_NOTE_A21']=='0':
    df_dep_2021_dis.append(QPV)
    missing_data_2021_dis += QPV.isna().sum()
    ind = -1

  elif ind == -1 and codgeo[3:5] != dep :
    break

#### Les données manquantes

In [ ]:
# Evolution des données manquantes au fil des années dans le département selectionné

missing_data_dis = [missing_data_2017_dis, missing_data_2018_dis, missing_data_2019_dis, missing_data_2020_dis, missing_data_2021_dis]

plt.figure(figsize=(16, 6))

plt.plot(annee, missing_data_dis)

plt.title("Données manquantes du département " + dep + " par année", fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("Nombre de données manquantes", fontsize=12)
plt.xticks(rotation=45, fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Le revenu disponible moyen par unité de consommation (quartile)

In [ ]:
# Récupération des données permettant d'observer l'évolution des revenus disponibles par quartiles
import statistics

# 2017
premier_quartile_2017 = []
mediane_2017 = []
troisieme_quartile_2017 = []

for QPV in df_dep_2017_dis:
  if pd.notna(QPV.iloc[1]):
    valeur = QPV.iloc[1].replace('\xa0', '').replace(' ', '')
    premier_quartile_2017.append(int(valeur))

  if pd.notna(QPV.iloc[2]):
    valeur = QPV.iloc[2].replace('\xa0', '').replace(' ', '')
    mediane_2017.append(int(valeur))

  if pd.notna(QPV.iloc[3]):
    valeur = QPV.iloc[3].replace('\xa0', '').replace(' ', '')
    troisieme_quartile_2017.append(int(valeur))


# 2018
premier_quartile_2018 = []
mediane_2018 = []
troisieme_quartile_2018 = []
for QPV in df_dep_2018_dis:
  if pd.notna(QPV.iloc[1]):
    valeur = QPV.iloc[1].replace('\xa0', '').replace(' ', '')
    premier_quartile_2018.append(int(valeur))

  if pd.notna(QPV.iloc[2]):
    valeur = QPV.iloc[2].replace('\xa0', '').replace(' ', '')
    mediane_2018.append(int(valeur))

  if pd.notna(QPV.iloc[3]):
    valeur = QPV.iloc[3].replace('\xa0', '').replace(' ', '')
    troisieme_quartile_2018.append(int(valeur))


# 2019
premier_quartile_2019 = []
mediane_2019 = []
troisieme_quartile_2019 = []
for QPV in df_dep_2019_dis:
  if pd.notna(QPV.iloc[1]):
    valeur = QPV.iloc[1].replace('\xa0', '').replace(' ', '')
    premier_quartile_2019.append(int(valeur))

  if pd.notna(QPV.iloc[2]):
    valeur = QPV.iloc[2].replace('\xa0', '').replace(' ', '')
    mediane_2019.append(int(valeur))

  if pd.notna(QPV.iloc[3]):
    valeur = QPV.iloc[3].replace('\xa0', '').replace(' ', '')
    troisieme_quartile_2019.append(int(valeur))

# 2020
premier_quartile_2020 = []
mediane_2020 = []
troisieme_quartile_2020 = []
for QPV in df_dep_2020_dis:
  if pd.notna(QPV.iloc[1]):
    valeur = QPV.iloc[1].replace('\xa0', '').replace(' ', '')
    premier_quartile_2020.append(int(valeur))

  if pd.notna(QPV.iloc[2]):
    valeur = QPV.iloc[2].replace('\xa0', '').replace(' ', '')
    mediane_2020.append(int(valeur))

  if pd.notna(QPV.iloc[3]):
    valeur = QPV.iloc[3].replace('\xa0', '').replace(' ', '')
    troisieme_quartile_2020.append(int(valeur))


# 2021
premier_quartile_2021 = []
mediane_2021 = []
troisieme_quartile_2021 = []
for QPV in df_dep_2021_dis:
  if pd.notna(QPV.iloc[1]):
    valeur = QPV.iloc[1].replace('\xa0', '').replace(' ', '')
    premier_quartile_2021.append(int(valeur))

  if pd.notna(QPV.iloc[2]):
    valeur = QPV.iloc[2].replace('\xa0', '').replace(' ', '')
    mediane_2021.append(int(valeur))

  if pd.notna(QPV.iloc[3]):
    valeur = QPV.iloc[3].replace('\xa0', '').replace(' ', '')
    troisieme_quartile_2021.append(int(valeur))


quartile_1_dep = [statistics.mean(premier_quartile_2017), statistics.mean(premier_quartile_2018), statistics.mean(premier_quartile_2019), statistics.mean(premier_quartile_2020), statistics.mean(premier_quartile_2021)]
mediane_dep = [statistics.mean(mediane_2017),statistics.mean(mediane_2018), statistics.mean(mediane_2019), statistics.mean(mediane_2020), statistics.mean(mediane_2021)]
quartile_3_dep = [statistics.mean(troisieme_quartile_2017), statistics.mean(troisieme_quartile_2018), statistics.mean(troisieme_quartile_2019), statistics.mean(troisieme_quartile_2020), statistics.mean(troisieme_quartile_2021)]


In [ ]:
# Affichage du graphique nous montrant l'évolution des revenus disponibles du département selectionné

plt.figure(figsize=(16, 6))

plt.plot(annee, quartile_1_dep, label ='1er quartile')
plt.plot(annee, mediane_dep, label = 'médiane')
plt.plot(annee, quartile_3_dep, label = '3ème quartile')

plt.title("Les revenus disponible moyen du département " + dep, fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("Revenu (en euros)", fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.legend()

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Le revenu disponible moyen par unité de consommation (décile)

In [ ]:
# Récupération des données permettant d'observer l'évolution des revenus disponibles par décile

# résultats
D1 = []
D2 = []
D3 = []
D4 = []
D6 = []
D7 = []
D8 = []
D9 = []


# temporaire
decile_1 = []
decile_2 = []
decile_3 = []
decile_4 = []
decile_6 = []
decile_7 = []
decile_8 = []
decile_9 = []

# 2017

for QPV in df_dep_2017_dis:
  if pd.notna(QPV.iloc[5]):
    valeur = QPV.iloc[5].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(QPV.iloc[6]):
    valeur = QPV.iloc[6].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(QPV.iloc[7]):
    valeur = QPV.iloc[7].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(QPV.iloc[8]):
    valeur = QPV.iloc[8].replace('\xa0', '').replace(' ', '')
    decile_4.append(int(valeur))

  if pd.notna(QPV.iloc[9]):
    valeur = QPV.iloc[9].replace('\xa0', '').replace(' ', '')
    decile_6.append(int(valeur))

  if pd.notna(QPV.iloc[10]):
    valeur = QPV.iloc[10].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(QPV.iloc[11]):
    valeur = QPV.iloc[11].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(QPV.iloc[12]):
    valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

D1.append(statistics.mean(decile_1))
D2.append(statistics.mean(decile_2))
D3.append(statistics.mean(decile_3))
D4.append(statistics.mean(decile_4))
D6.append(statistics.mean(decile_6))
D7.append(statistics.mean(decile_7))
D8.append(statistics.mean(decile_8))
D9.append(statistics.mean(decile_9))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_4.clear()
decile_6.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()



# 2018
premier_quartile_2018 = []
mediane_2018 = []
troisieme_quartile_2018 = []
for QPV in df_dep_2018_dis:
  if pd.notna(QPV.iloc[5]):
    valeur = QPV.iloc[5].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(QPV.iloc[6]):
    valeur = QPV.iloc[6].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(QPV.iloc[7]):
    valeur = QPV.iloc[7].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(QPV.iloc[8]):
    valeur = QPV.iloc[8].replace('\xa0', '').replace(' ', '')
    decile_4.append(int(valeur))

  if pd.notna(QPV.iloc[9]):
    valeur = QPV.iloc[9].replace('\xa0', '').replace(' ', '')
    decile_6.append(int(valeur))

  if pd.notna(QPV.iloc[10]):
    valeur = QPV.iloc[10].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(QPV.iloc[11]):
    valeur = QPV.iloc[11].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(QPV.iloc[12]):
    valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

D1.append(statistics.mean(decile_1))
D2.append(statistics.mean(decile_2))
D3.append(statistics.mean(decile_3))
D4.append(statistics.mean(decile_4))
D6.append(statistics.mean(decile_6))
D7.append(statistics.mean(decile_7))
D8.append(statistics.mean(decile_8))
D9.append(statistics.mean(decile_9))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_4.clear()
decile_6.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()


# 2019
premier_quartile_2019 = []
mediane_2019 = []
troisieme_quartile_2019 = []
for QPV in df_dep_2019_dis:
  if pd.notna(QPV.iloc[5]):
    valeur = QPV.iloc[5].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(QPV.iloc[6]):
    valeur = QPV.iloc[6].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(QPV.iloc[7]):
    valeur = QPV.iloc[7].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(QPV.iloc[8]):
    valeur = QPV.iloc[8].replace('\xa0', '').replace(' ', '')
    decile_4.append(int(valeur))

  if pd.notna(QPV.iloc[9]):
    valeur = QPV.iloc[9].replace('\xa0', '').replace(' ', '')
    decile_6.append(int(valeur))

  if pd.notna(QPV.iloc[10]):
    valeur = QPV.iloc[10].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(QPV.iloc[11]):
    valeur = QPV.iloc[11].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(QPV.iloc[12]):
    valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

D1.append(statistics.mean(decile_1))
D2.append(statistics.mean(decile_2))
D3.append(statistics.mean(decile_3))
D4.append(statistics.mean(decile_4))
D6.append(statistics.mean(decile_6))
D7.append(statistics.mean(decile_7))
D8.append(statistics.mean(decile_8))
D9.append(statistics.mean(decile_9))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_4.clear()
decile_6.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()

# 2020
premier_quartile_2020 = []
mediane_2020 = []
troisieme_quartile_2020 = []
for QPV in df_dep_2020_dis:
  if pd.notna(QPV.iloc[5]):
    valeur = QPV.iloc[5].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(QPV.iloc[6]):
    valeur = QPV.iloc[6].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(QPV.iloc[7]):
    valeur = QPV.iloc[7].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(QPV.iloc[8]):
    valeur = QPV.iloc[8].replace('\xa0', '').replace(' ', '')
    decile_4.append(int(valeur))

  if pd.notna(QPV.iloc[9]):
    valeur = QPV.iloc[9].replace('\xa0', '').replace(' ', '')
    decile_6.append(int(valeur))

  if pd.notna(QPV.iloc[10]):
    valeur = QPV.iloc[10].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(QPV.iloc[11]):
    valeur = QPV.iloc[11].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(QPV.iloc[12]):
    valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

D1.append(statistics.mean(decile_1))
D2.append(statistics.mean(decile_2))
D3.append(statistics.mean(decile_3))
D4.append(statistics.mean(decile_4))
D6.append(statistics.mean(decile_6))
D7.append(statistics.mean(decile_7))
D8.append(statistics.mean(decile_8))
D9.append(statistics.mean(decile_9))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_4.clear()
decile_6.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()


# 2021
premier_quartile_2021 = []
mediane_2021 = []
troisieme_quartile_2021 = []
for QPV in df_dep_2021_dis:
  if pd.notna(QPV.iloc[5]):
    valeur = QPV.iloc[5].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(QPV.iloc[6]):
    valeur = QPV.iloc[6].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(QPV.iloc[7]):
    valeur = QPV.iloc[7].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(QPV.iloc[8]):
    valeur = QPV.iloc[8].replace('\xa0', '').replace(' ', '')
    decile_4.append(int(valeur))

  if pd.notna(QPV.iloc[9]):
    valeur = QPV.iloc[9].replace('\xa0', '').replace(' ', '')
    decile_6.append(int(valeur))

  if pd.notna(QPV.iloc[10]):
    valeur = QPV.iloc[10].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(QPV.iloc[11]):
    valeur = QPV.iloc[11].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(QPV.iloc[12]):
    valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

D1.append(statistics.mean(decile_1))
D2.append(statistics.mean(decile_2))
D3.append(statistics.mean(decile_3))
D4.append(statistics.mean(decile_4))
D6.append(statistics.mean(decile_6))
D7.append(statistics.mean(decile_7))
D8.append(statistics.mean(decile_8))
D9.append(statistics.mean(decile_9))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_4.clear()
decile_6.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()

In [ ]:
# Affichage du graphique permettant d'observer l'évolution des revenus disponibles par déciles

plt.figure(figsize=(16, 6))

plt.plot(annee, D1, label='1er décile')
plt.plot(annee, D2, label='2ème décile')
plt.plot(annee, D3, label='3ème décile')
plt.plot(annee, D4, label='4ème décile')
plt.plot(annee, D6, label='6ème décile')
plt.plot(annee, D7, label='7ème décile')
plt.plot(annee, D8, label='8ème décile')
plt.plot(annee, D9, label='9ème décile')

plt.title("Les revenus disponible moyen du département " + dep, fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("Revenu (en euros)", fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.legend()

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Indice de gini (les inégalités)

In [ ]:
# Récupération des données permettant d'observer l'évolution des inégalités dans le département

import statistics

G = []
gini = []

# 2017
for QPV in df_dep_2017_dis:
  if pd.notna(QPV.iloc[14]):
    valeur = QPV.iloc[14].replace(',', '.')
    gini.append(float(valeur))

G.append(statistics.mean(gini))
gini.clear()


# 2018
for QPV in df_dep_2018_dis:
  if pd.notna(QPV.iloc[14]):
    valeur = QPV.iloc[14].replace(',', '.')
    gini.append(float(valeur))

G.append(statistics.mean(gini))
gini.clear()


# 2019
for QPV in df_dep_2019_dis:
  if pd.notna(QPV.iloc[14]):
    valeur = QPV.iloc[14].replace(',', '.')
    gini.append(float(valeur))

G.append(statistics.mean(gini))
gini.clear()


# 2020
for QPV in df_dep_2020_dis:
  if pd.notna(QPV.iloc[14]):
    valeur = QPV.iloc[14].replace(',', '.')
    gini.append(float(valeur))

G.append(statistics.mean(gini))
gini.clear()


# 2021
for QPV in df_dep_2021_dis:
  if pd.notna(QPV.iloc[14]):
    valeur = QPV.iloc[14].replace(',', '.')
    gini.append(float(valeur))

G.append(statistics.mean(gini))
gini.clear()

In [ ]:
# Affichage du graphique montrant l'évolution des inégalités sur le revenu disponible au sein des QPV dans le département selectionné

plt.figure(figsize=(16, 6))

plt.plot(annee, G)

plt.title("Le niveau d'inégalité du département " + dep, fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("indice de Gini", fontsize=12)
plt.xticks(rotation=45, fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### Carte des départements de la Métrople

#### Initialisation des données

In [ ]:
# Récupération de la carte des départements en format geojson

# Télécharger les données GeoJSON
url = "https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb"
response = requests.get(url)
departements_geojson = response.json()


#### Récupération des données sur les revenus disponibles des 10 % les plus aisés des qpv par département

##### 2017

In [ ]:
# Affichage de la carte des départements métroplitains selon le revenu disponible des 10% les plus aisés des QPV de chaque département pour l'année 2017

from branca.colormap import StepColormap

# Comptage du nombre de département par catégorie
niv_1 = [0,0,0,0,0]
niv_2 = [0,0,0,0,0]
niv_3 = [0,0,0,0,0]
niv_4 = [0,0,0,0,0]
niv_5 = [0,0,0,0,0]
niv_6 = [0,0,0,0,0]


# Récupérer la moyenne du 9ème décile des départements de la métropole
dict_dep_aises = {}
for i in range(2,len(df_pauvrete_2017_dis['codgeo'])):
  QPV = df_pauvrete_2017_dis.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[2] != '0':
    break
  dict_dep_aises.setdefault(codgeo[3:5], []).append(int(QPV.iloc[12]))

dict_mean_dep_aises_2017 = {cle: sum(valeurs)/len(valeurs) for cle, valeurs in dict_dep_aises.items()}


# Fonction pour mapper une couleur en fonction de la valeur
def get_color(value):
    if value < 19000:
      niv_1[0] += 1
      return '#006400'
    elif value < 21500:
      niv_2[0] += 1
      return '#80ff00'
    elif value < 25000:
      niv_3[0] += 1
      return '#ffff00'
    elif value < 28500:
      niv_4[0] += 1
      return '#ffbf00'
    elif value < 31000:
      niv_5[0] += 1
      return '#ff8000'
    else:
      niv_6[0] += 1
      return '#ff0000'

# Style en fonction des données
def style_function(feature):
    code = feature["properties"]["code"]  # Code département
    value = int(dict_mean_dep_aises_2017.get(code, 0))  # Récupère la valeur ou 0 par défaut
    return {
        'fillColor': get_color(value),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.8,
    }

# Créer une carte centrée sur la France
map_dept_revenu_2017 = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter les départements
folium.GeoJson(departements_geojson, name="Départements").add_to(map_dept_revenu_2017)

folium.GeoJson(
    departements_geojson,
    name="Départements",
    style_function=style_function
).add_to(map_dept_revenu_2017)


layer_2017 = folium.FeatureGroup(name='Carte 2017')

legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 175px;
    height: 160px;
    background-color: #ededed;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
">
    <b>Revenu médian (€) :</b><br>
    <i style="background: #006400; width: 10px; height: 10px; display: inline-block;"></i> < 19 000<br>
    <i style="background: #80ff00; width: 10px; height: 10px; display: inline-block;"></i> 19 000 - 21 500<br>
    <i style="background: #ffff00; width: 10px; height: 10px; display: inline-block;"></i> 21 500 - 25 000<br>
    <i style="background: #ffbf00; width: 10px; height: 10px; display: inline-block;"></i> 25 000 - 28 500<br>
    <i style="background: #ff8000; width: 10px; height: 10px; display: inline-block;"></i> 28 500 - 31 000<br>
    <i style="background: #ff0000; width: 10px; height: 10px; display: inline-block;"></i> > 31 000<br>
</div>
"""
map_dept_revenu_2017.get_root().html.add_child(folium.Element(legend_html))

# Afficher la carte
map_dept_revenu_2017

##### 2018

In [ ]:
# Affichage de la carte des départements métroplitains selon le revenu disponible des 10% les plus aisés des QPV de chaque département pour l'année 2018

from branca.colormap import StepColormap

# Récupérer la moyenne du 9ème décile des départements de la métropole
dict_dep_aises = {}
for i in range(2,len(df_pauvrete_2018_dis['CODGEO'])):
  QPV = df_pauvrete_2018_dis.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[2] != '0':
    break
  valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
  dict_dep_aises.setdefault(codgeo[3:5], []).append(int(valeur))

dict_mean_dep_aises_2018 = {cle: sum(valeurs)/len(valeurs) for cle, valeurs in dict_dep_aises.items()}


# Fonction pour mapper une couleur en fonction de la valeur
def get_color(value):
    if value < 19000:
      niv_1[1] += 1
      return '#006400'
    elif value < 21500:
      niv_2[1] += 1
      return '#80ff00'
    elif value < 25000:
      niv_3[1] += 1
      return '#ffff00'
    elif value < 28500:
      niv_4[1] += 1
      return '#ffbf00'
    elif value < 31000:
      niv_5[1] += 1
      return '#ff8000'
    else:
      niv_6[1] += 1
      return '#ff0000'

# Style en fonction des données
def style_function(feature):
    code = feature["properties"]["code"]  # Code département
    value = int(dict_mean_dep_aises_2018.get(code, 0))  # Récupère la valeur ou 0 par défaut
    return {
        'fillColor': get_color(value),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.8,
    }

# Créer une carte centrée sur la France
map_dept_revenu_2018 = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter les départements
folium.GeoJson(departements_geojson, name="Départements").add_to(map_dept_revenu_2018)

folium.GeoJson(
    departements_geojson,
    name="Départements",
    style_function=style_function
).add_to(map_dept_revenu_2018)


layer_2018 = folium.FeatureGroup(name='Carte 2018')

legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 175px;
    height: 160px;
    background-color: #ededed;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
">
    <b>Revenu médian (€) :</b><br>
    <i style="background: #006400; width: 10px; height: 10px; display: inline-block;"></i> < 19 000<br>
    <i style="background: #80ff00; width: 10px; height: 10px; display: inline-block;"></i> 19 000 - 21 500<br>
    <i style="background: #ffff00; width: 10px; height: 10px; display: inline-block;"></i> 21 500 - 25 000<br>
    <i style="background: #ffbf00; width: 10px; height: 10px; display: inline-block;"></i> 25 000 - 28 500<br>
    <i style="background: #ff8000; width: 10px; height: 10px; display: inline-block;"></i> 28 500 - 31 000<br>
    <i style="background: #ff0000; width: 10px; height: 10px; display: inline-block;"></i> > 31 000<br>
</div>
"""
map_dept_revenu_2018.get_root().html.add_child(folium.Element(legend_html))

# Afficher la carte
map_dept_revenu_2018

##### 2019

In [ ]:
# Affichage de la carte des départements métroplitains selon le revenu disponible des 10% les plus aisés des QPV de chaque département pour l'année 2019

from branca.colormap import StepColormap

# Récupérer la moyenne du 9ème décile des départements de la métropole
dict_dep_aises = {}
for i in range(2,len(df_pauvrete_2019_dis['CODGEO'])):
  QPV = df_pauvrete_2019_dis.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[2] != '0':
    break
  valeur = QPV.iloc[12].replace('\xa0', '').replace(' ', '')
  dict_dep_aises.setdefault(codgeo[3:5], []).append(int(valeur))

dict_mean_dep_aises_2019 = {cle: sum(valeurs)/len(valeurs) for cle, valeurs in dict_dep_aises.items()}


# Fonction pour mapper une couleur en fonction de la valeur
def get_color(value):
    if value < 19000:
      niv_1[2] += 1
      return '#006400'
    elif value < 21500:
      niv_2[2] += 1
      return '#80ff00'
    elif value < 25000:
      niv_3[2] += 1
      return '#ffff00'
    elif value < 28500:
      niv_4[2] += 1
      return '#ffbf00'
    elif value < 31000:
      niv_5[2] += 1
      return '#ff8000'
    else:
      niv_6[2] += 1
      return '#ff0000'

# Style en fonction des données
def style_function(feature):
    code = feature["properties"]["code"]  # Code département
    value = int(dict_mean_dep_aises_2019.get(code, 0))  # Récupère la valeur ou 0 par défaut
    return {
        'fillColor': get_color(value),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.8,
    }

# Créer une carte centrée sur la France
map_dept_revenu_2019 = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter les départements
folium.GeoJson(departements_geojson, name="Départements").add_to(map_dept_revenu_2019)

folium.GeoJson(
    departements_geojson,
    name="Départements",
    style_function=style_function
).add_to(map_dept_revenu_2019)


layer_2019 = folium.FeatureGroup(name='Carte 2019')

legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 175px;
    height: 160px;
    background-color: #ededed;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
">
    <b>Revenu médian (€) :</b><br>
    <i style="background: #006400; width: 10px; height: 10px; display: inline-block;"></i> < 19 000<br>
    <i style="background: #80ff00; width: 10px; height: 10px; display: inline-block;"></i> 19 000 - 21 500<br>
    <i style="background: #ffff00; width: 10px; height: 10px; display: inline-block;"></i> 21 500 - 25 000<br>
    <i style="background: #ffbf00; width: 10px; height: 10px; display: inline-block;"></i> 25 000 - 28 500<br>
    <i style="background: #ff8000; width: 10px; height: 10px; display: inline-block;"></i> 28 500 - 31 000<br>
    <i style="background: #ff0000; width: 10px; height: 10px; display: inline-block;"></i> > 31 000<br>
</div>
"""
map_dept_revenu_2019.get_root().html.add_child(folium.Element(legend_html))

# Afficher la carte
map_dept_revenu_2019

##### 2020

In [ ]:
# Affichage de la carte des départements métroplitains selon le revenu disponible des 10% les plus aisés des QPV de chaque département pour l'année 2020

from branca.colormap import StepColormap

# Récupérer la moyenne du 9ème décile des départements de la métropole
dict_dep_aises = {}
for i in range(2,len(df_pauvrete_2020_dis['CODGEO'])):
  QPV = df_pauvrete_2020_dis.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[2] != '0':
    break
  valeur = QPV.iloc[12]
  if pd.notna(valeur):
    valeur = valeur.replace('\xa0', '').replace(' ', '')
  else:
    valeur = '0'
  if valeur != '':
    dict_dep_aises.setdefault(codgeo[3:5], []).append(int(valeur))

dict_mean_dep_aises_2020 = {cle: sum(valeurs)/len(valeurs) for cle, valeurs in dict_dep_aises.items()}


# Fonction pour mapper une couleur en fonction de la valeur
def get_color(value):
    if value == 0:
      return '#808080'
    elif value < 19000:
      niv_1[3] += 1
      return '#006400'
    elif value < 21500:
      niv_2[3] += 1
      return '#80ff00'
    elif value < 25000:
      niv_3[3] += 1
      return '#ffff00'
    elif value < 28500:
      niv_4[3] += 1
      return '#ffbf00'
    elif value < 31000:
      niv_5[3] += 1
      return '#ff8000'
    else:
      niv_6[3] += 1
      return '#ff0000'

# Style en fonction des données
def style_function(feature):
    code = feature["properties"]["code"]  # Code département
    value = int(dict_mean_dep_aises_2020.get(code, 0))  # Récupère la valeur ou 0 par défaut
    return {
        'fillColor': get_color(value),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.8,
    }

# Créer une carte centrée sur la France
map_dept_revenu_2020 = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter les départements
folium.GeoJson(departements_geojson, name="Départements").add_to(map_dept_revenu_2020)

folium.GeoJson(
    departements_geojson,
    name="Départements",
    style_function=style_function
).add_to(map_dept_revenu_2020)

layer_2020 = folium.FeatureGroup(name='Carte 2020')

legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 175px;
    height: 190px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
">
    <b>Revenu médian (€) :</b><br>
    <i style="background: #808080; width: 10px; height: 10px; display: inline-block;"></i> manque de données<br>
    <i style="background: #006400; width: 10px; height: 10px; display: inline-block;"></i> < 19 000<br>
    <i style="background: #80ff00; width: 10px; height: 10px; display: inline-block;"></i> 19 000 - 21 500<br>
    <i style="background: #ffff00; width: 10px; height: 10px; display: inline-block;"></i> 21 500 - 25 000<br>
    <i style="background: #ffbf00; width: 10px; height: 10px; display: inline-block;"></i> 25 000 - 28 500<br>
    <i style="background: #ff8000; width: 10px; height: 10px; display: inline-block;"></i> 28 500 - 31 000<br>
    <i style="background: #ff0000; width: 10px; height: 10px; display: inline-block;"></i> > 31 000<br>
</div>
"""
map_dept_revenu_2020.get_root().html.add_child(folium.Element(legend_html))

# Afficher la carte
map_dept_revenu_2020

##### 2021

In [ ]:
# Affichage de la carte des départements métroplitains selon le revenu disponible des 10% les plus aisés des QPV de chaque département pour l'année 2021

from branca.colormap import StepColormap


# Récupérer la moyenne du 9ème décile des départements de la métropole
dict_dep_aises = {}
for i in range(2,len(df_pauvrete_2021_dis['CODGEO'])):
  QPV = df_pauvrete_2021_dis.iloc[i]
  codgeo = QPV.iloc[0]
  if codgeo[2] != '0':
    break
  valeur = QPV.iloc[12]
  if pd.notna(valeur):
    valeur = valeur.replace('\xa0', '').replace(' ', '')
  else:
    valeur = '0'
  if valeur != '':
    dict_dep_aises.setdefault(codgeo[3:5], []).append(int(valeur))

dict_mean_dep_aises_2021 = {cle: sum(valeurs)/len(valeurs) for cle, valeurs in dict_dep_aises.items()}


# Fonction pour mapper une couleur en fonction de la valeur
def get_color(value):
    if value == 0:
      return '#808080'
    elif value < 19000:
      niv_1[4] += 1
      return '#006400'
    elif value < 21500:
      niv_2[4] += 1
      return '#80ff00'
    elif value < 25000:
      niv_3[4] += 1
      return '#ffff00'
    elif value < 28500:
      niv_4[4] += 1
      return '#ffbf00'
    elif value < 31000:
      niv_5[4] += 1
      return '#ff8000'
    else:
      niv_6[4] += 1
      return '#ff0000'

# Style en fonction des données
def style_function(feature):
    code = feature["properties"]["code"]  # Code département
    value = int(dict_mean_dep_aises_2021.get(code, 0))  # Récupère la valeur ou 0 par défaut
    return {
        'fillColor': get_color(value),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.8,
    }

# Créer une carte centrée sur la France
map_dept_revenu_2021 = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter les départements
folium.GeoJson(departements_geojson, name="Départements").add_to(map_dept_revenu_2021)

folium.GeoJson(
    departements_geojson,
    name="Départements",
    style_function=style_function
).add_to(map_dept_revenu_2021)

layer_2021 = folium.FeatureGroup(name='Carte 2021')

legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 175px;
    height: 190px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
">
    <b>Revenu médian (€) :</b><br>
    <i style="background: #808080; width: 10px; height: 10px; display: inline-block;"></i> manque de données<br>
    <i style="background: #006400; width: 10px; height: 10px; display: inline-block;"></i> < 19 000<br>
    <i style="background: #80ff00; width: 10px; height: 10px; display: inline-block;"></i> 19 000 - 21 500<br>
    <i style="background: #ffff00; width: 10px; height: 10px; display: inline-block;"></i> 21 500 - 25 000<br>
    <i style="background: #ffbf00; width: 10px; height: 10px; display: inline-block;"></i> 25 000 - 28 500<br>
    <i style="background: #ff8000; width: 10px; height: 10px; display: inline-block;"></i> 28 500 - 31 000<br>
    <i style="background: #ff0000; width: 10px; height: 10px; display: inline-block;"></i> > 31 000<br>
</div>
"""
map_dept_revenu_2021.get_root().html.add_child(folium.Element(legend_html))

# Afficher la carte
map_dept_revenu_2021

#### Ensemble des années

In [ ]:
# Affichage des cartes de façon superposée (problème au niveau des données de 2021)

from branca.colormap import StepColormap
import folium
from folium import plugins
from branca.element import Template, MacroElement

# Création de la carte de base
map_france = folium.Map(location=[46.603354, 1.888334], zoom_start=6)


# Définir une fonction pour obtenir les couleurs en fonction des valeurs (comme précédemment)
def get_color(value):
    if value < 19000:
        return '#006400'
    elif value < 21500:
        return '#80ff00'
    elif value < 25000:
        return '#ffff00'
    elif value < 28500:
        return '#ffbf00'
    elif value < 31000:
        return '#ff8000'
    else:
        return '#ff0000'

# Exemple de données pour 5 années différentes
years = [2017, 2018, 2019, 2020, 2021]
data_by_year = {
    2017: dict_mean_dep_aises_2017,
    2018: dict_mean_dep_aises_2018,
    2019: dict_mean_dep_aises_2019,
    2020: dict_mean_dep_aises_2020,
    2021: dict_mean_dep_aises_2021
}

# Créer une fonction pour ajouter des GeoJson en fonction de l'année
geojson_layers = []
for year in years:
    # Ici, ajouter le GeoJson pour chaque année. Remplacez `departements_geojson` par vos propres données GeoJSON.
    geojson = folium.GeoJson(
        departements_geojson,  # Remplacez par vos données GeoJSON des départements
        name=f"Départements {year}",
        style_function=lambda feature, year=year: {
            'fillColor': get_color(data_by_year[year].get(feature["properties"]["code"], 0)),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 1
        }
    )
    geojson_layers.append(geojson)

# Ajouter les couches à la carte mais les désactiver initialement
for geojson in geojson_layers:
    geojson.add_to(map_france)

# Créer une fonction pour afficher les différentes couches selon l'année choisie
def add_button_for_year(year, map_obj):
    button_html = f"""
    <div style="position: fixed; bottom: 10px; left: 10px; background-color: white;
    padding: 10px; font-size: 16px; z-index: 1000;">
        <button onclick="document.getElementById('map-layer-{year}').style.display='block';">Afficher {year}</button>
    </div>
    """
    # Ajouter un bouton sur la carte pour chaque année
    map_obj.get_root().html.add_child(folium.Element(button_html))

    # Ajout du contrôle pour l'année spécifique
    geojson_layers[years.index(year)].add_to(map_obj)




# Ajouter le contrôle pour afficher ou masquer les différentes couches
folium.LayerControl().add_to(map_france)

legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 175px;
    height: 190px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
">
    <b>Revenu médian (€) :</b><br>
    <i style="background: #808080; width: 10px; height: 10px; display: inline-block;"></i> manque de données<br>
    <i style="background: #006400; width: 10px; height: 10px; display: inline-block;"></i> < 19 000<br>
    <i style="background: #80ff00; width: 10px; height: 10px; display: inline-block;"></i> 19 000 - 21 500<br>
    <i style="background: #ffff00; width: 10px; height: 10px; display: inline-block;"></i> 21 500 - 25 000<br>
    <i style="background: #ffbf00; width: 10px; height: 10px; display: inline-block;"></i> 25 000 - 28 500<br>
    <i style="background: #ff8000; width: 10px; height: 10px; display: inline-block;"></i> 28 500 - 31 000<br>
    <i style="background: #ff0000; width: 10px; height: 10px; display: inline-block;"></i> > 31 000<br>
</div>
"""
map_france.get_root().html.add_child(folium.Element(legend_html))

# Afficher la carte
map_france


#### Graphique sur les évolutions par année

In [ ]:
# Récupération des données permettant de visualiser l'évolution des revenus disponibles par déciles en France

import statistics

# Initialisation des données
decile_1 = []
decile_2 = []
decile_3 = []
decile_7 = []
decile_8 = []
decile_9 = []

decile_qpv_1 = []
decile_qpv_2 = []
decile_qpv_3 = []
decile_qpv_7 = []
decile_qpv_8 = []
decile_qpv_9 = []

gini = []
indice_G_mediane = []
indice_G_moyenne = []

# 2017
for index, row in df_pauvrete_2017_dis.iterrows():
  # Je veux seulement la France métropolitaine donc j'effectue un premier trie
  codgeo = row['codgeo']
  if codgeo[2]!='0' and codgeo[2]!='s':
    break

  if pd.notna(row['disp_d1_a17']):
    valeur = row['disp_d1_a17'].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(row['disp_d2_a17']):
    valeur = row['disp_d2_a17'].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(row['disp_d3_a17']):
    valeur = row['disp_d3_a17'].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(row['disp_d7_a17']):
    valeur = row['disp_d7_a17'].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(row['disp_d8_a17']):
    valeur = row['disp_d8_a17'].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(row['disp_d9_a17']):
    valeur = row['disp_d9_a17'].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

  if pd.notna(row['disp_gini_a17']):
    valeur = row['disp_gini_a17'].replace('\xa0', '').replace(' ', '')
    gini.append(float(valeur.replace(',', '.')))

decile_qpv_1.append(statistics.median(decile_1))
decile_qpv_2.append(statistics.median(decile_2))
decile_qpv_3.append(statistics.median(decile_3))
decile_qpv_7.append(statistics.median(decile_7))
decile_qpv_8.append(statistics.median(decile_8))
decile_qpv_9.append(statistics.median(decile_9))

indice_G_mediane.append(statistics.median(gini))
indice_G_moyenne.append(statistics.mean(gini))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()
gini.clear()


# 2018
for index, row in df_pauvrete_2018_dis.iterrows():
  # Je veux seulement la France métropolitaine donc j'effectue un premier trie
  codgeo = row['CODGEO']
  if codgeo[2]!='0' and codgeo[2]!='s':
    break

  if pd.notna(row['DISP_D1_A18']):
    valeur = row['DISP_D1_A18'].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(row['DISP_D2_A18']):
    valeur = row['DISP_D2_A18'].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(row['DISP_D3_A18']):
    valeur = row['DISP_D3_A18'].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(row['DISP_D7_A18']):
    valeur = row['DISP_D7_A18'].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(row['DISP_D8_A18']):
    valeur = row['DISP_D8_A18'].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(row['DISP_D9_A18']):
    valeur = row['DISP_D9_A18'].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

  if pd.notna(row['DISP_GINI_A18']):
    valeur = row['DISP_GINI_A18'].replace('\xa0', '').replace(' ', '')
    gini.append(float(valeur.replace(',', '.')))

decile_qpv_1.append(statistics.median(decile_1))
decile_qpv_2.append(statistics.median(decile_2))
decile_qpv_3.append(statistics.median(decile_3))
decile_qpv_7.append(statistics.median(decile_7))
decile_qpv_8.append(statistics.median(decile_8))
decile_qpv_9.append(statistics.median(decile_9))

indice_G_mediane.append(statistics.median(gini))
indice_G_moyenne.append(statistics.mean(gini))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()
gini.clear()

# 2019
for index, row in df_pauvrete_2019_dis.iterrows():
  # Je veux seulement la France métropolitaine donc j'effectue un premier trie
  codgeo = row['CODGEO']
  if codgeo[2]!='0' and codgeo[2]!='s':
    break

  if pd.notna(row['DISP_D1_A19']):
    valeur = row['DISP_D1_A19'].replace('\xa0', '').replace(' ', '')
    decile_1.append(int(valeur))

  if pd.notna(row['DISP_D2_A19']):
    valeur = row['DISP_D2_A19'].replace('\xa0', '').replace(' ', '')
    decile_2.append(int(valeur))

  if pd.notna(row['DISP_D3_A19']):
    valeur = row['DISP_D3_A19'].replace('\xa0', '').replace(' ', '')
    decile_3.append(int(valeur))

  if pd.notna(row['DISP_D7_A19']):
    valeur = row['DISP_D7_A19'].replace('\xa0', '').replace(' ', '')
    decile_7.append(int(valeur))

  if pd.notna(row['DISP_D8_A19']):
    valeur = row['DISP_D8_A19'].replace('\xa0', '').replace(' ', '')
    decile_8.append(int(valeur))

  if pd.notna(row['DISP_D9_A19']):
    valeur = row['DISP_D9_A19'].replace('\xa0', '').replace(' ', '')
    decile_9.append(int(valeur))

  if pd.notna(row['DISP_GINI_A19']):
    valeur = row['DISP_GINI_A19'].replace('\xa0', '').replace(' ', '')
    gini.append(float(valeur.replace(',', '.')))

decile_qpv_1.append(statistics.median(decile_1))
decile_qpv_2.append(statistics.median(decile_2))
decile_qpv_3.append(statistics.median(decile_3))
decile_qpv_7.append(statistics.median(decile_7))
decile_qpv_8.append(statistics.median(decile_8))
decile_qpv_9.append(statistics.median(decile_9))

indice_G_mediane.append(statistics.median(gini))
indice_G_moyenne.append(statistics.mean(gini))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()
gini.clear()

# 2020
for index, row in df_pauvrete_2020_dis.iterrows():
  # Je veux seulement la France métropolitaine donc j'effectue un premier trie
  codgeo = row['CODGEO']
  if codgeo[2]!='0' and codgeo[2]!='s':
    break

  if pd.notna(row['DISP_D1_A20']):
    valeur = row['DISP_D1_A20'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_1.append(int(valeur))

  if pd.notna(row['DISP_D2_A20']):
    valeur = row['DISP_D2_A20'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_2.append(int(valeur))

  if pd.notna(row['DISP_D3_A20']):
    valeur = row['DISP_D3_A20'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_3.append(int(valeur))

  if pd.notna(row['DISP_D7_A20']):
    valeur = row['DISP_D7_A20'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_7.append(int(valeur))

  if pd.notna(row['DISP_D8_A20']):
    valeur = row['DISP_D8_A20'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_8.append(int(valeur))

  if pd.notna(row['DISP_D9_A20']):
    valeur = row['DISP_D9_A20'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_9.append(int(valeur))

  if pd.notna(row['DISP_GINI_A20']):
    valeur = row['DISP_GINI_A20'].replace('\xa0', '').replace(' ', '')
    if valeur:
      gini.append(float(valeur.replace(',', '.')))

decile_qpv_1.append(statistics.median(decile_1))
decile_qpv_2.append(statistics.median(decile_2))
decile_qpv_3.append(statistics.median(decile_3))
decile_qpv_7.append(statistics.median(decile_7))
decile_qpv_8.append(statistics.median(decile_8))
decile_qpv_9.append(statistics.median(decile_9))

indice_G_mediane.append(statistics.median(gini))
indice_G_moyenne.append(statistics.mean(gini))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()
gini.clear()


# 2021
for index, row in df_pauvrete_2021_dis.iterrows():
  # Je veux seulement la France métropolitaine donc j'effectue un premier trie
  codgeo = row['CODGEO']
  if codgeo[2]!='0' and codgeo[2]!='s':
    break

  if pd.notna(row['DISP_D1_A21']):
    valeur = row['DISP_D1_A21'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_1.append(int(valeur))

  if pd.notna(row['DISP_D2_A21']):
    valeur = row['DISP_D2_A21'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_2.append(int(valeur))

  if pd.notna(row['DISP_D3_A21']):
    valeur = row['DISP_D3_A21'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_3.append(int(valeur))

  if pd.notna(row['DISP_D7_A21']):
    valeur = row['DISP_D7_A21'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_7.append(int(valeur))

  if pd.notna(row['DISP_D8_A21']):
    valeur = row['DISP_D8_A21'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_8.append(int(valeur))

  if pd.notna(row['DISP_D9_A21']):
    valeur = row['DISP_D9_A21'].replace('\xa0', '').replace(' ', '')
    if valeur:
      decile_9.append(int(valeur))

  if pd.notna(row['DISP_GINI_A21']):
    valeur = row['DISP_GINI_A21'].replace('\xa0', '').replace(' ', '')
    if valeur:
      gini.append(float(valeur.replace(',', '.')))

decile_qpv_1.append(statistics.median(decile_1))
decile_qpv_2.append(statistics.median(decile_2))
decile_qpv_3.append(statistics.median(decile_3))
decile_qpv_7.append(statistics.median(decile_7))
decile_qpv_8.append(statistics.median(decile_8))
decile_qpv_9.append(statistics.median(decile_9))

indice_G_mediane.append(statistics.median(gini))
indice_G_moyenne.append(statistics.mean(gini))

decile_1.clear()
decile_2.clear()
decile_3.clear()
decile_7.clear()
decile_8.clear()
decile_9.clear()
gini.clear()

In [ ]:
# Affichage du graphique des revenus disponibles par décile récupéré précédemment

import matplotlib.pyplot as plt

annees = ["2017", "2018", "2019", "2020", "2021"]

plt.figure(figsize=(16, 6))  # Augmenter la largeur de la figure

plt.plot(annees, decile_qpv_1, label='1er décile')
plt.plot(annees, decile_qpv_2, label='2ème décile')
plt.plot(annees, decile_qpv_3, label='3ème décile')
plt.plot(annees, decile_qpv_7, label='7ème décile')
plt.plot(annees, decile_qpv_8, label='8ème décile')
plt.plot(annees, decile_qpv_9, label='9ème décile')

plt.title("Le revenu disponible médian des QPV", fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("Revenu (en euros)", fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.legend()

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Visualisation de l'augmentation des revenus disponibles par décile en pourcentage de 2017 à 2021

# 1er décile
pourc_1 = (decile_qpv_1[4] - decile_qpv_1[0])*100 / decile_qpv_1[0]

# 2ème décile
pourc_2 = (decile_qpv_2[4] - decile_qpv_2[0])*100 / decile_qpv_2[0]

# 3ème décile
pourc_3 = (decile_qpv_3[4] - decile_qpv_3[0])*100 / decile_qpv_3[0]

# 7ème décile
pourc_7 = (decile_qpv_7[4] - decile_qpv_7[0])*100 / decile_qpv_7[0]

# 8ème décile
pourc_8 = (decile_qpv_8[4] - decile_qpv_8[0])*100 / decile_qpv_8[0]

# 9ème décile
pourc_9 = (decile_qpv_9[4] - decile_qpv_9[0])*100 / decile_qpv_9[0]


import matplotlib.pyplot as plt

deciles = ["1", "2", "3", "7", "8", "9"]

fig, ax = plt.subplots(figsize=(8, 6))

bars = ax.barh(deciles, [pourc_1, pourc_2, pourc_3, pourc_7, pourc_8, pourc_9], color='skyblue')

for bar in bars:
    value = bar.get_width()
    # Ajouter le texte à la fin de chaque barre
    ax.text(value, bar.get_y() + bar.get_height()/2,
            f'{value:.2f}%',
            va='center',
            ha='left',
            fontsize=10,
            color='black')

ax.set_title("L'augmentation du revenu médian des QPV de 2017 à 2021", fontsize=14)
ax.set_xlabel("augmentation (en %)", fontsize=12)
ax.set_ylabel("Déciles", fontsize=12)

plt.show()


In [ ]:
# Observation de l'indice de Gini (les inégalités) en France au sein des QPV

import matplotlib.pyplot as plt

annees = ["2017", "2018", "2019", "2020", "2021"]

plt.figure(figsize=(16, 6))

plt.plot(annees, indice_G_moyenne, label='Moyenne')
plt.plot(annees, indice_G_mediane, label='Médianne')

plt.title("L'indice de gini des QPV par année", fontsize=14)
plt.xlabel("Année", fontsize=12)
plt.ylabel("indice de Gini", fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.legend()

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Observer le pourcentage d'augmentation au niveau des revenus par décile au sein des QPV

# Pour le premier décile
evol_1 = decile_qpv_1[4] - decile_qpv_1[0]
pourc_1 = evol_1 * 100 / decile_qpv_1[0]

# Pour le 2ème décile
evol_2 = decile_qpv_2[4] - decile_qpv_2[0]
pourc_2 = evol_2 * 100 / decile_qpv_2[0]

# Pour le 3ème décile
evol_3 = decile_qpv_3[4] - decile_qpv_3[0]
pourc_3 = evol_3 * 100 / decile_qpv_3[0]

# Pour le 7ème décile
evol_7 = decile_qpv_7[4] - decile_qpv_7[0]
pourc_7 = evol_7 * 100 / decile_qpv_7[0]

# Pour le 8ème décile
evol_8 = decile_qpv_8[4] - decile_qpv_8[0]
pourc_8 = evol_8 * 100 / decile_qpv_8[0]

# Pour le 9ème décile décile
evol_9 = decile_qpv_9[4] - decile_qpv_9[0]
pourc_9 = evol_9 * 100 / decile_qpv_9[0]

print("1er décile : augmentation de ", evol_1, " euros soit ", round(pourc_1, 2), "% d'augmentation")
print("2ème décile : augmentation de ", evol_2, " euros soit ", round(pourc_2, 2), "% d'augmentation")
print("3ème décile : augmentation de ", evol_3, " euros soit ", round(pourc_3, 2), "% d'augmentation")
print("7ème décile : augmentation de ", evol_7, " euros soit ", round(pourc_7, 2), "% d'augmentation")
print("8ème décile : augmentation de ", evol_8, " euros soit ", round(pourc_8, 2), "% d'augmentation")
print("9ème décile : augmentation de ", evol_9, " euros soit ", round(pourc_9, 2), "% d'augmentation")

In [ ]:
# Graphique comparant les revenus médians disponibles entre le niveau national et les QPV
# Récupération des données manuellement sur le site data gouv pour le revenu médian national français par année et calcul préalablement réalisé pour le revenu médian au sein des QPV

import matplotlib.pyplot as plt
import numpy as np

annees = ("2017", "2018", "2019", "2020", "2021")
penguin_means = {
    'QPV': (13600.0, 13850.0, 14070.0, 14875.0, 14955.0),
    'National': (23040.0, 23100.0, 23690.0, 24470.0, 24360.0)
}

x = np.arange(len(annees))
width = 0.25
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

for attribute, measurement in penguin_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1


ax.set_ylabel('Revenu disponible par unité de consommation')
ax.set_xlabel('Année')
ax.set_title('Revenu médian disponible par années')
ax.set_xticks(x + width, annees)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 27000)

plt.show()

#### Tentative d'affichage des QPV

In [ ]:
# Je suppose que les coordonnées présentes dans le fichier geojson sont mauvaises d'où le mauvais affichage

# Chemin vers le fichier GeoJSON dans votre Drive
geojson_path = '/content/drive/MyDrive/PIP2025_G7_Elections/Analyse approfondie/Analyse socio - Législatives/Données/Pauvreté/qp-2024-epsg2154-20240820.geojson'  # Remplacez par votre chemin exact

# Charger les données GeoJSON
with open(geojson_path, 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Extraire le premier quartier
first_feature = geojson_data['features'][0]

# Extraire les coordonnées du premier quartier
if first_feature['geometry']['type'] == 'Polygon':
    center_coords = first_feature['geometry']['coordinates'][0][0]
elif first_feature['geometry']['type'] == 'MultiPolygon':
    center_coords = first_feature['geometry']['coordinates'][0][0][0]
else:
    raise ValueError("Le type de géométrie n'est pas pris en charge.")

center_coords = center_coords[::-1]

# Créer une carte centrée sur le premier quartier
m = folium.Map(location=center_coords, zoom_start=15)

# Ajouter uniquement le premier quartier à la carte
folium.GeoJson(
    first_feature,
    name="Premier Quartier"
).add_to(m)

folium.LayerControl().add_to(m)

# Afficher la carte
m

## Travail réalisé autour de la Haute-Garonne

### Carte des bureaux de votes de la Haute-Garonne

In [ ]:

# Chemin vers le fichier GeoJSON
geojson_path = "/content/drive/MyDrive/PIP2025_G7_Elections/Codes personnels/Code Morgan/departement_31 - Haute-Garonne.geojson"

communes = gpd.read_file(geojson_path)
map_center = [43.6, 1.4]
map_communes = folium.Map(location=map_center, zoom_start=9)
folium.GeoJson(
    communes,
    name="Bureaux des communes",
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.5,
    },
).add_to(map_communes)

# Afficher la carte
map_communes


### Carte des communes de la Haute Garonne par revenus médians

#### Récupération des données

In [ ]:
# Chemin vers le fichier CSV
csv_path = "/content/drive/MyDrive/PIP2025_G7_Elections/Codes personnels/Code Morgan/FILO2021_DISP_COM.csv"

# Charger le fichier CSV dans un DataFrame
df_commune_metropole = pd.read_csv(csv_path, sep = ";")

# Chemin vers le fichier CSV (nom des communes)
csv_path = "/content/drive/MyDrive/PIP2025_G7_Elections/Codes personnels/Code Morgan/meta_FILO2021_DISP_COM.csv"

# Charger le fichier CSV dans un DataFrame
df_commune_metropole_meta = pd.read_csv(csv_path, sep = ";")

In [ ]:
dict_codgeo_dept_communes = {'codgeo': [], 'commune' : [], 'dept' : []}

# Itérer sur chaque ligne du DataFrame
for index, row in df_commune_metropole_meta.iterrows():
    # Extraire le code du département
    if row['COD_VAR'] == 'CODGEO':
      codgeo_str = str(row['COD_MOD'])
      if len(codgeo_str) == 4:
        code_dept = '0' + codgeo_str[0]
      else:
        code_dept = codgeo_str[:2]

      if (code_dept == '97'):
        break

      # Ajouter ce qu'il faut (code géographique, nom de la commune)
      dict_codgeo_dept_communes['codgeo'].append(codgeo_str)
      dict_codgeo_dept_communes['commune'].append(row['LIB_MOD'])
      dict_codgeo_dept_communes['dept'].append(code_dept)

In [ ]:
df_codgeo_dept_communes = pd.DataFrame(dict_codgeo_dept_communes)
df_codgeo_revenu_communes = df_commune_metropole[['CODGEO','Q221','NBPERS21']]

# Convertir les CODGEO en chaînes et ajouter un zéro si la longueur est 4
df_codgeo_revenu_communes['CODGEO'] = df_codgeo_revenu_communes['CODGEO'].astype(str).apply(lambda x: '0' + x if len(x) == 4 else x)

df_carte_revenu = pd.merge(df_codgeo_dept_communes, df_codgeo_revenu_communes, left_on='codgeo', right_on='CODGEO', how='inner')

df_carte_revenu['Q221'] = df_carte_revenu['Q221'].replace(['s', 'N/A', 'unknown', 'nd'], 0)

In [ ]:
# Récupérer le fichier contenant le nom des communes de chaque code géo
df_carte_revenu['Q221'] = df_carte_revenu['Q221'].replace(['s', 'N/A', 'unknown', 'nd','None'], 0)

df_carte_revenu_Haute_Garonne = df_carte_revenu[df_carte_revenu['dept']=="31"]
df_carte_revenu_Haute_Garonne

#### Affichage de la carte

In [ ]:
# Charger le GeoJSON
geojson_path = '/content/drive/MyDrive/PIP2025_G7_Elections/Codes personnels/Code Morgan/a-com2022.json'
gdf = gpd.read_file(geojson_path)


In [ ]:
# Fusionner les données géographiques avec les revenus médians
gdf = gdf.merge(df_carte_revenu, on='codgeo', how='left')

# Filtrer les communes du département 31
gdf_31 = gdf[gdf['codgeo'].str.startswith('31')]  # Filtrer pour le département 31
geojson_data = json.loads(gdf_31.to_json())

In [ ]:
# Définir les couleurs
def get_color(revenu):
    revenu = int(revenu)
    if revenu == 0:  # Pas de donnée
        return '#d3d3d3'  # Gris clair
    elif revenu < 14200 :
        return '#FFFF00'
    elif revenu < 22200:
        return '#fee8c8'  # Orange clair
    elif revenu < 27000:
        return '#fdbb84'  # Orange moyen
    elif revenu < 35000:
        return '#e34a33'  # Orange foncé
    else:
        return '#b30000'  # Rouge

# Appliquer un style basé sur les revenus
def style_function(feature):
    revenu = feature['properties'].get('Q221', 0)
    return {
        'fillColor': get_color(revenu),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    }

# Créer une carte centrée sur la Haute-Garonne
map_communes = folium.Map(location=[43.604, 1.444], zoom_start=9)

# Ajouter les communes avec un style basé sur le revenu médian
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['commune', 'Q221', 'NBPERS21'],
        aliases=['Commune:', 'Revenu médian (€):', 'Nbr de personnes :'],
        localize=True
    )
).add_to(map_communes)

# Afficher la carte
map_communes


### Carte de la Haute-Garonne par circonscription

#### Récupération des données

In [ ]:
geojson_path = '/content/drive/MyDrive/PIP2025_G7_Elections/Codes personnels/Code Morgan/circonscriptions-legislatives-p10.geojson'
gdf = gpd.read_file(geojson_path)

In [ ]:
# Créer mon data frame contenent le département et le nom de la circonscription
dict_circonscription_Haute_Garonne = {'code_cir' : [], 'nom_cir' : [], 'revenu_median' : [], 'taux_pauvrete' : []}


In [ ]:
# Chemin vers le fichier Excel
file_path = "/content/drive/MyDrive/PIP2025_G7_Elections/Codes personnels/Code Morgan/indic-stat-circonscriptions-legislatives-2022.xlsx"

# Lire le fichier Excel dans un DataFrame
df_circonscription = pd.read_excel(file_path, sheet_name="indicateurs_circonscriptions", header=7)

In [ ]:
for index, row in df_circonscription.iterrows():
  code_cir = row['circo']
  if code_cir[0:2]=='31':
    code_cir_map = code_cir[0:2] + code_cir[3:5]
    dict_circonscription_Haute_Garonne['code_cir'].append(code_cir_map)
    dict_circonscription_Haute_Garonne['nom_cir'].append(row['Nom de la circonscription'])
    dict_circonscription_Haute_Garonne['revenu_median'].append(row['nivvie_median_diff'])
    dict_circonscription_Haute_Garonne['taux_pauvrete'].append(row['tx_pauvrete60_diff'])

df_circonscription_Haute_Garonne = pd.DataFrame(dict_circonscription_Haute_Garonne)

In [ ]:
# Récupérer les données GeoJSON à partir de l'URL
url_geojson = "https://www.data.gouv.fr/fr/datasets/r/8b681b69-739c-47eb-a96b-06e8e2d8dc08"
response = requests.get(url_geojson)
geojson_data = response.json()

In [ ]:
gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])
gdf_31 = gdf[gdf['codeCirconscription'].str.startswith('31')]
gdf_31 = gdf_31.merge(df_circonscription_Haute_Garonne, left_on='codeCirconscription', right_on='code_cir', how='left')
geojson_data_updated = json.loads(gdf_31.to_json())

#### Affichage de la carte (revenu médian)

In [ ]:
# Définir les couleurs
def get_color(revenu):
    revenu = int(revenu)
    if revenu == 0:
        return '#ededed'
    elif revenu < 21000:
        return '#ffffcc'
    elif revenu < 22000:
        return '#a1dab4'
    elif revenu < 23000:
        return '#41b6c4'
    elif revenu < 26000:
        return '#225ea8'
    else :
        return '#130c4d'

# Appliquer un style basé sur les revenus
def style_function(feature):
    revenu = feature['properties'].get('revenu_median', 0)
    return {
        'fillColor': get_color(revenu),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    }


map_communes = folium.Map(location=[43.604, 1.444], zoom_start=9)

folium.GeoJson(
    geojson_data_updated,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['nom_cir', 'revenu_median'],
        aliases=['Nom :', 'Revenu médian (€):'],
        localize=True
    )
).add_to(map_communes)

legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 175px;
    height: 150px;
    background-color: #ededed;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
">
    <b>Revenu médian (€) :</b><br>
    <i style="background: #ffffcc; width: 10px; height: 10px; display: inline-block;"></i> < 21 000<br>
    <i style="background: #a1dab4; width: 10px; height: 10px; display: inline-block;"></i> 21 000 - 22 000<br>
    <i style="background: #41b6c4; width: 10px; height: 10px; display: inline-block;"></i> 22 000 - 23 000<br>
    <i style="background: #225ea8; width: 10px; height: 10px; display: inline-block;"></i> 23 000 - 26 000<br>
    <i style="background: #130c4d; width: 10px; height: 10px; display: inline-block;"></i> > 26 000<br>
</div>
"""
map_communes.get_root().html.add_child(folium.Element(legend_html))

map_communes.save('/content/drive/MyDrive/PIP2025_G7_Elections/Codes personnels/Code Morgan/map_circonscription_Haute_Garonne_revenu.html')

# Afficher la carte
map_communes

#### Affichage de la carte (taux de pauvreté)

In [ ]:
# Définir les couleurs
def get_color(revenu):
    value = int(revenu)
    if value == 0:
        return '#ededed'
    elif value < 10:
        return '#130c4d'
    elif value < 14:
        return '#225ea8'
    elif value < 18:
        return '#41b6c4'
    elif value < 20:
        return '#a1dab4'
    else:
        return '#ffffcc'

def style_function(feature):
    revenu = feature['properties'].get('taux_pauvrete', 0)
    return {
        'fillColor': get_color(revenu),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    }

map_communes = folium.Map(location=[43.604, 1.444], zoom_start=9)

folium.GeoJson(
    geojson_data_updated,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['nom_cir', 'taux_pauvrete'],
        aliases=['Nom :', 'taux de pauvreté (%):'],
        localize=True
    )
).add_to(map_communes)

legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 175px;
    height: 150px;
    background-color: #ededed;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
">
    <b>Taux de pauvreté (%) :</b><br>
    <i style="background: #130c4d; width: 10px; height: 10px; display: inline-block;"></i> < 10<br>
    <i style="background: #225ea8; width: 10px; height: 10px; display: inline-block;"></i> 10 - 14<br>
    <i style="background: #41b6c4; width: 10px; height: 10px; display: inline-block;"></i> 14 - 18<br>
    <i style="background: #a1dab4; width: 10px; height: 10px; display: inline-block;"></i> 18 - 20<br>
    <i style="background: #ffffcc; width: 10px; height: 10px; display: inline-block;"></i> > 20<br>
</div>
"""
map_communes.get_root().html.add_child(folium.Element(legend_html))

map_communes.save('/content/drive/MyDrive/PIP2025_G7_Elections/Codes personnels/Code Morgan/map_circonscription_Haute_Garonne_taux.html')

# Afficher la carte
map_communes

### Graphique comparant taux de pauvreté et votes en Haute-Garonne

In [ ]:
file_path = "/content/drive/MyDrive/PIP2025_G7_Elections/Codes personnels/Code Morgan/resultats-par-niveau-cirlg-t1-france-entiere.xlsx"

# Lire le fichier Excel
df_resultats_leg = pd.read_excel(file_path)

df_resultats_leg_HG = df_resultats_leg[df_resultats_leg['Code du département']=='31']

In [ ]:
dict_t1 = []
dict_t2 = []
dict_t3 = []
dict_t4 = []
dict_t5 = []

for index, row in df_resultats_leg_HG.iterrows():
  code_cir = row['Code de la circonscription']
  if (code_cir == 5 or code_cir == 6 or code_cir == 10):
    ind = 23
    while(ind < 215):
      parti = row.iloc[ind]
      vote = row.iloc[ind+1]
      if parti:
        dict_t1.append((parti, vote))
        ind+=9

  if (code_cir == 3 or code_cir == 7):
    ind = 23
    while(ind < 215):
      parti = row.iloc[ind]
      vote = row.iloc[ind+1]
      if parti is None:
        break
      dict_t2.append((parti, vote))
      ind+=9

  if (code_cir == 8 or code_cir == 1 or code_cir == 2):
    ind = 23
    while(ind < 215):
      parti = row.iloc[ind]
      vote = row.iloc[ind+1]
      if parti:
        dict_t3.append((parti, vote))
        ind+=9

  if (code_cir == 9):
    ind = 23
    while(ind < 215):
      parti = row.iloc[ind]
      vote = row.iloc[ind+1]
      if parti == 'nan':
        print("debug")
      dict_t4.append((parti, vote))
      ind+=9

  if (code_cir == 4):
    ind = 23
    while(ind < 215):
      parti = row.iloc[ind]
      vote = row.iloc[ind+1]
      if parti is None:
        break
      dict_t5.append((parti, vote))
      ind+=9


In [ ]:
# Pour le premier taux de pauvreté
gauche = [0,0,0,0,0]
extr_gauche = [0,0,0,0,0]
droite = [0,0,0,0,0]
extr_droite = [0,0,0,0,0]
centre = [0,0,0,0,0]
divers = [0,0,0,0,0]

for i in range(len(dict_t1)):
  parti = dict_t1[i][0]
  vote = dict_t1[i][1]
  match parti:
    # extreme gauche
    case "EXG" | "DXG":
      extr_gauche[0]+=int(vote)
    # gauche
    case "FI" | "DVG" | "COM" | "SOC" | "RDG" | "NUP" | "UG" | "VEC" :
      gauche[0]+=int(vote)
    # centre
    case "MDM" | "REM" | "ENS" | "DVC" | "HOR" :
      centre[0]+=int(vote)
    # droite
    case "UDI" | "LR" | "DVD":
      droite[0]+=int(vote)
    # extreme droite
    case "FN" | "DLF" | "EXD" | "DSV" | "REC" | "RN" | "DXD" | "UXD":
      extr_droite[0]+=int(vote)
    # divers
    case "DIV" | "ECO" | "REG":
      divers[0]+=int(vote)

# Pour le premier taux de pauvreté
taux_t2 = [0,0,0,0,0,0]
for i in range(len(dict_t2)):
  parti = dict_t2[i][0]
  vote = dict_t2[i][1]
  match parti:
    # extreme gauche
    case "EXG" | "DXG":
      extr_gauche[1]+=int(vote)
    # gauche
    case "FI" | "DVG" | "COM" | "SOC" | "RDG" | "NUP" | "UG" | "VEC" :
      gauche[1]+=int(vote)
    # centre
    case "MDM" | "REM" | "ENS" | "DVC" | "HOR" :
      centre[1]+=int(vote)
    # droite
    case "UDI" | "LR" | "DVD":
      droite[1]+=int(vote)
    # extreme droite
    case "FN" | "DLF" | "EXD" | "DSV" | "REC" | "RN" | "DXD" | "UXD":
      extr_droite[1]+=int(vote)
    # divers
    case "DIV" | "ECO" | "REG":
      divers[1]+=int(vote)

# Pour le premier taux de pauvreté
taux_t3 = [0,0,0,0,0,0]
for i in range(len(dict_t3)):
  parti = dict_t3[i][0]
  vote = dict_t3[i][1]
  match parti:
    # extreme gauche
    case "EXG" | "DXG":
      extr_gauche[2]+=int(vote)
    # gauche
    case "FI" | "DVG" | "COM" | "SOC" | "RDG" | "NUP" | "UG" | "VEC" :
      gauche[2]+=int(vote)
    # centre
    case "MDM" | "REM" | "ENS" | "DVC" | "HOR" :
      centre[2]+=int(vote)
    # droite
    case "UDI" | "LR" | "DVD":
      droite[2]+=int(vote)
    # extreme droite
    case "FN" | "DLF" | "EXD" | "DSV" | "REC" | "RN" | "DXD" | "UXD":
      extr_droite[2]+=int(vote)
    # divers
    case "DIV" | "ECO" | "REG":
      divers[2]+=int(vote)

# Pour le premier taux de pauvreté
taux_t4 = [0,0,0,0,0,0]
for i in range(len(dict_t4)):
  parti = dict_t4[i][0]
  vote = dict_t4[i][1]
  match parti:
    # extreme gauche
    case "EXG" | "DXG":
      extr_gauche[3]+=int(vote)
    # gauche
    case "FI" | "DVG" | "COM" | "SOC" | "RDG" | "NUP" | "UG" | "VEC" :
      gauche[3]+=int(vote)
    # centre
    case "MDM" | "REM" | "ENS" | "DVC" | "HOR" :
      centre[3]+=int(vote)
    # droite
    case "UDI" | "LR" | "DVD":
      droite[3]+=int(vote)
    # extreme droite
    case "FN" | "DLF" | "EXD" | "DSV" | "REC" | "RN" | "DXD" | "UXD":
      extr_droite[3]+=int(vote)
    # divers
    case "DIV" | "ECO" | "REG":
      divers[3]+=int(vote)

# Pour le premier taux de pauvreté
taux_t5 = [0,0,0,0,0,0]
for i in range(len(dict_t5)):
  parti = dict_t5[i][0]
  vote = dict_t5[i][1]
  match parti:
    # extreme gauche
    case "EXG" | "DXG":
      extr_gauche[4]+=int(vote)
    # gauche
    case "FI" | "DVG" | "COM" | "SOC" | "RDG" | "NUP" | "UG" | "VEC" :
      gauche[4]+=int(vote)
    # centre
    case "MDM" | "REM" | "ENS" | "DVC" | "HOR" :
      centre[4]+=int(vote)
    # droite
    case "UDI" | "LR" | "DVD":
      droite[4]+=int(vote)
    # extreme droite
    case "FN" | "DLF" | "EXD" | "DSV" | "REC" | "RN" | "DXD" | "UXD":
      extr_droite[4]+=int(vote)
    # divers
    case "DIV" | "ECO" | "REG":
      divers[4]+=int(vote)

In [ ]:
total_t1 = sum([extr_gauche[0], gauche[0], droite[0], extr_droite[0], centre[0], divers[0]])
total_t2 = sum([extr_gauche[1], gauche[1], droite[1], extr_droite[1], centre[1], divers[1]])
total_t3 = sum([extr_gauche[2], gauche[2], droite[2], extr_droite[2], centre[2], divers[2]])
total_t4 = sum([extr_gauche[3], gauche[3], droite[3], extr_droite[3], centre[3], divers[3]])
total_t5 = sum([extr_gauche[4], gauche[4], droite[4], extr_droite[4], centre[4], divers[4]])


for i in range(5):
  if i == 0:
    total = total_t1
  elif i == 1:
    total = total_t2
  elif i == 2:
    total = total_t3
  elif i == 3:
    total = total_t4
  elif i == 4:
    total = total_t5
  gauche[i] = gauche[i]*100/total
  extr_gauche[i] = extr_gauche[i]*100/total
  droite[i] = droite[i]*100/total
  extr_droite[i] = extr_droite[i]*100/total
  centre[i] = centre[i]*100/total
  divers[i] = divers[i]*100/total

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# Données
categories = ['< 10', '10-14', '14-18', '18-20', '> 20']

# Positions des barres
x = np.arange(len(categories))

# Création du graphique
plt.bar(x, extr_gauche, label='Extreme gauche', color='blue')
plt.bar(x, gauche, bottom=extr_gauche, label='Gauche', color='orange')
plt.bar(x, centre, bottom=np.array(extr_gauche) + np.array(gauche), label='Centre', color='red')
plt.bar(x, droite, bottom=np.array(extr_gauche) + np.array(gauche) + np.array(centre), label='Droite', color='green')
plt.bar(x, extr_droite, bottom=np.array(extr_gauche) + np.array(gauche) + np.array(centre) + np.array(droite), label='Extreme droite', color='black')
plt.bar(x, divers, bottom=np.array(extr_gauche) + np.array(gauche) + np.array(centre) + np.array(droite) + np.array(extr_droite), label='Divers', color='yellow')

# Ajout des labels
plt.xticks(x, categories)
plt.xlabel('Catégories')
plt.ylabel('Valeurs')
plt.title('Graphique en barres empilées')
plt.legend()

# Affichage
plt.show()